# TODO 

- Data from 2000-2010
- Hourly values of PV-Pot
    - Find min/max
    - Daily averages
    - Weekly averages
- Sum up over europes landmass
-  Rank it by the highest - Take x many highest cells that are needed
    - this will kind of already give the area 
- Take mean over Europe?? take mean over Gridcells in Europe?
    - 1000km sidelenght find minimum area? Is it consistent?
    - Get the means in the Gridcells hourly/weekly
    - Compare it throughout the years
          - Does the grid with pv change
          - Values change?

## Get area for worst case! 

In [1]:
import xarray as xr
import geopandas as gpd

In [2]:
data_path = "/Users/yhra/Documents/Master/Semester_3/ICM/offline_data/"
shapefile_path = "/Users/yhra/Documents/Master/Semester_3/ICM/Europe_Task/europe_10km.shx"

In [3]:
from core import clip_to_europe

original_ds = xr.open_dataset(data_path + "era5-2000-01.nc",
                                     engine="netcdf4",
                                     chunks={"valid_time":1e5})

clipped_ds = clip_to_europe(original_ds, shapefile_path)

clipped_ds

/Users/yhra/Documents/Master/Semester_3/ICM/Europe_Task/core.py:48: UserWarning: Shapefile appears to be in projected coordinates. Attempting to reproject...
  warnings.warn("Shapefile appears to be in projected coordinates. Attempting to reproject...")


<xarray.Dataset>
Dimensions:      (valid_time: 744, latitude: 280, longitude: 715)
Coordinates:
    number       int64 ...
  * valid_time   (valid_time) datetime64[ns] 2000-01-01 ... 2000-01-31T23:00:00
  * latitude     (latitude) float64 89.75 89.5 89.25 89.0 ... 20.5 20.25 20.0
  * longitude    (longitude) float64 0.0 0.25 0.5 0.75 ... 178.0 178.2 178.5
    expver       (valid_time) object dask.array<chunksize=(744,), meta=np.ndarray>
    spatial_ref  int64 0
Data variables:
    t2m          (valid_time, latitude, longitude) float32 dask.array<chunksize=(744, 280, 715), meta=np.ndarray>
    ssrd         (valid_time, latitude, longitude) float32 dask.array<chunksize=(744, 280, 715), meta=np.ndarray>
    ssrdc        (valid_time, latitude, longitude) float32 dask.array<chunksize=(744, 280, 715), meta=np.ndarray>
    u10          (valid_time, latitude, longitude) float32 dask.array<chunksize=(744, 280, 715), meta=np.ndarray>
    v10          (valid_time, latitude, longitude) float32 dask.array<chunksize=(744, 280, 715), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-21T19:50 GRIB to CDM+CF via cfgrib-0.9.1...